In [11]:
!pip install kaggle
!pip install hdfs

     |████████████████████████████████| 58 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 78 kB 3.5 MB/s  eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=0e10c450b56a8051e5d7c01f4aa5a26f7f0dcbd93a2a1fac5d7fbdd0a3ff8ff0
  Stored in directory: /root/.cache/pip/wheels/77/47/e4/44a4ba1b7dfd53faaa35f59f1175e123b213ff401a8a56876b
Successfully built kaggle
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=0317ab5e7721ecbb296160eb68e7e916c0e9ce6e1df2e150b35deb6a4474b7fd
  Stored in directory: /root/.cache/pip/wheels/3f/2a/fa/4d7a888e69774d5e6e855d190a8a51b357d77cc05eb1c097c9
Successfully built docopt


# Kaggle configuration, API Token

In [6]:
import os
base_path = "/mnt/notebooks/Desafio1_FIAP"
os.environ["KAGGLE_CONFIG_DIR"] = f'{base_path}/kaggle_config_dir/'
!chmod 600 /mnt/notebooks/Desafio1_FIAP/kaggle_config_dir/kaggle.json

# Download dataset files from Kaggle

In [3]:
import kaggle
kaggle.api.authenticate()

kaggle.api.dataset_download_files('olistbr/brazilian-ecommerce', 
                                  path='/mnt/notebooks/Individual_Desafio1_FIAP/olist_dataset', 
                                  unzip=True)

# Ingest dataset files to HDFS

## Parâmetros de conexão

In [12]:
import requests
import os
import pandas as pd 
import hdfs
import urllib3

from hdfs import InsecureClient
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util import Retry

max_threads = 50
session = requests.Session()

retry_strategy = Retry(
    total=10,
    connect=10,
    read=10,
    redirect=10,
    status_forcelist=[429, 500, 502, 503, 504],
    method_whitelist=["HEAD", "GET", "OPTIONS"],
)

adapter = HTTPAdapter(
    max_retries=retry_strategy, pool_connections=max_threads, pool_maxsize=max_threads,
)

session.mount("https://", adapter)
session.mount("http://", adapter)

# client usando IP do host docker
client = 'http://192.168.56.1:50070'

# Client HDFS
hdfs_client = InsecureClient(client, session=session)

## Escrevendo na landing zone HDFS

In [23]:
# Gravar o arquivo csv no HDFS
for filename in os.listdir(f'{base_path}/olist_dataset'):
    if filename == 'olist_customers_dataset.csv':
        df = pd.read_csv(f'{base_path}/olist_dataset/{filename}', sep =',', dtype={'customer_zip_code_prefix':'object'})
    elif filename == 'olist_sellers_dataset.csv':
        df = pd.read_csv(f'{base_path}/olist_dataset/{filename}', sep =',', dtype={'seller_zip_code_prefix':'object'})
    elif filename == 'olist_geolocation_dataset.csv':
        df = pd.read_csv(f'{base_path}/olist_dataset/{filename}', sep =',', dtype={'geolocation_zip_code_prefix':'object'})
    else:
        df = pd.read_csv(f'{base_path}/olist_dataset/{filename}', sep =',')
    df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
    try:
        with hdfs_client.write(f'/datalake/landing_zone/{filename}', overwrite = True, encoding='utf-8') as writer:
            df.to_csv(writer, header=True, index=False)
        print(f"{filename} Gravado com sucesso")
    except hdfs.util.HdfsError as e:
        print(f"{filename} falhou")
        print(f"[ERRO] {e}")
    except urllib3.exceptions.NewConnectionError as e:
        print(f"{filename} falhou")
        print(f"[ERRO] {e}")
    except Exception as e:
        print(e)

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix    object
customer_city               object
customer_state              object
dtype: object
olist_customers_dataset.csv Gravado com sucesso
geolocation_zip_code_prefix     object
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object
olist_geolocation_dataset.csv Gravado com sucesso
olist_orders_dataset.csv Gravado com sucesso
olist_order_items_dataset.csv Gravado com sucesso
olist_order_payments_dataset.csv Gravado com sucesso
olist_order_reviews_dataset.csv Gravado com sucesso
olist_products_dataset.csv Gravado com sucesso
seller_id                 object
seller_zip_code_prefix    object
seller_city               object
seller_state              object
dtype: object
olist_sellers_dataset.csv Gravado com sucesso
product_category_name_translation.csv Gravado com sucesso


## Transformando arquivos de csv para orc

In [24]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Ingest Olist Dataset") \
    .getOrCreate()

landing_zone = '/datalake/landing_zone/'
files = hdfs_client.list(landing_zone)

for filename in files:
    csv = spark.read.csv(f'{landing_zone}/{filename}', header = True, sep = ',')
    orc_name = filename.replace('csv', 'orc')
    csv.write.orc(f'/datalake/dadosbrutos/{orc_name}', 'overwrite')
    